In [1]:
# import importlib
# import local_utils; importlib.reload(local_utils)
from local_utils import *

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

/opt/conda/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Seed: 7961730


In [2]:
# Load Data
ids, comments, Y, test_ids, test_comments, inx2label, label2inx = load_data(True)
Y_wblank = np.concatenate([Y, np.expand_dims((~Y.any(axis=1)).astype(int), 1)], axis=1)
print("Original:\n" + comments[0])
print()

comments = Parallel(n_jobs=cpu_cores)(delayed(preprocess)(text, False) for text in comments)
test_comments = Parallel(n_jobs=cpu_cores)(delayed(preprocess)(text, False) for text in test_comments)
print("Processed:\n" + comments[0])

# comments_fr, comments_de, comments_es = load_augmented_data()

Original:
explanation why the edits made under my username hardcore metallica fan were reverted? they weren't vandalisms, just closure on some gas after i voted at new york dolls fac. and please don't remove the template from the talk page since I'm retired now. 

Processed:
explanation why the edits made under my username hardcore metallica fan were reverted? they weren't vandalisms, just closure on some gas after i voted at new york dolls fac. and please don't remove the template from the talk page since I'm retired now. 


In [3]:
# def get_coefs(word,*arr):
#     return word, np.asarray(arr, dtype='float32')

# inx2word = []
# vectors = []
# for o in open('data/talk/vectors_utf8.txt', encoding='utf8'):
#     try:
#         w,v = get_coefs(*o.strip().split(' '))
#         if len(v) == 300:
#             inx2word.append(w)
#             vectors.append(v)
#     except:
#         pass

# vectors = np.stack(vectors)
# barr = bcolz.carray(vectors, rootdir="data/talk/glove-cheng-vectors.bcolz")
# barr.flush()

# pickle.dump(inx2word, open('data/talk/glove-cheng-words.pkl', 'wb'))

In [4]:
vectors, inx2word, word2inx = load_embs()
# vectors, inx2word, word2inx = load_embs(embs_dir='data/talk/', embs_name='glove-cheng')

In [5]:
# def tokenize(text):
#     return deepmoji_tokenizer.tokenize(text, word2inx)

# docs = Parallel(n_jobs=cpu_cores)(delayed(tokenize)(text) for text in comments + test_comments)
# pickle.dump(docs, open('data/tokenized_comments.pkl', 'wb'))

In [6]:
docs = pickle.load(open('data/tokenized_comments.pkl', 'rb'))

In [7]:
max_len = 450
text_analyzer = TextAnalyzer(word2inx, vectors, reverse=True, max_len=max_len, process_oov_words=True, oov_min_doc_hits=5)
seq, meta = text_analyzer.fit_on_docs(docs)

X = seq[:len(comments)]
test_X = seq[len(comments):]

meta_mean = meta.mean(axis=0)
meta_std = meta.std(axis=0)
meta = (meta - meta_mean)/meta_std

print("mean_len: {}".format(meta_mean[0]))
print("mean_len + 2*std: {}".format(meta_mean[0]+2*meta_std[0]))
print("mean_len + 3*std: {}".format(meta_mean[0]+3*meta_std[0]))

X_meta = meta[:len(comments)]
test_X_meta = meta[len(comments):]

Docs: 312735
Selected words: 172905
Processed OOV words: 4533
mean_len: 74.75018466113482
mean_len + 2*std: 313.1402682213584
mean_len + 3*std: 432.33531000147013


In [8]:
# Train/Valid splitting
trn_inx, val_inx = stratified_sampling(Y, 0.1, seed)

print("train: {}, valid: {}".format(len(trn_inx), len(val_inx)))
# plot_stratified_sampling(Y, trn_inx, val_inx, inx2label)

train: 143613, valid: 15958


In [9]:
# ros = RandomOverSampler(random_state=seed)
# trn_inx = ros.fit_sample(trn_inx.reshape(-1, 1), Y_wblank[trn_inx,6])[0].flatten()

# rus = RandomUnderSampler(return_indices=False)
# trn_inx = rus.fit_sample(trn_inx.reshape(-1, 1), Y_wblank[trn_inx,6])[0].flatten()

In [10]:
# fr_seq, fr_meta = text_analyzer.transform_texts(comments_fr)
# de_seq, de_meta = text_analyzer.transform_texts(comments_de)
# es_seq, es_meta = text_analyzer.transform_texts(comments_es)

# trn_X = np.concatenate([X[trn_inx], fr_seq[trn_inx], de_seq[trn_inx], es_seq[trn_inx]])
# trn_X_meta = np.concatenate([X_meta[trn_inx], fr_meta[trn_inx], de_meta[trn_inx], es_meta[trn_inx]])
# trn_Y = np.concatenate([Y[trn_inx], Y[trn_inx], Y[trn_inx], Y[trn_inx]])

In [11]:
# CNN
# def cnn_block(x, filters, kernel_size, attention=0):
#     cnn = Conv1D(filters=filters, kernel_size=kernel_size, activation='relu')(x)
    
#     if attention == 0: cnn = GlobalMaxPooling1D()(cnn)
#     elif attention == 1: cnn = AttentionWeightedAverage()(cnn)
#     elif attention == 2: cnn = Attention()(cnn)

#     return cnn

# def getCNNModel(input_shape, classes, num_words, emb_size, emb_matrix,
#                 attention=0, dense=False, emb_trainable=False):

#     x_input = Input(shape=(input_shape,))
    
#     emb = Embedding(num_words, emb_size, weights=[emb_matrix], trainable=emb_trainable, name='embs')(x_input)
#     emb = SpatialDropout1D(0.15)(emb)
        
#     cnn1 = cnn_block(emb, 100, 3, attention=attention)
#     cnn2 = cnn_block(emb, 100, 4, attention=attention)
#     cnn3 = cnn_block(emb, 100, 5, attention=attention)
#     x = concatenate([cnn1, cnn2, cnn3])

#     x = Dropout(0.15)(x)
    
#     if dense: 
#         x = Dense(50, activation='relu')(x)
#         x = Dropout(0.15)(x)
    
#     x_output = Dense(classes, activation='sigmoid')(x)
#     return Model(inputs=x_input, outputs=x_output)

In [12]:
# CNN-LSTM
def getCNNLSTMModel(input_shape, classes, num_words, emb_size, emb_matrix,
                    attention=0, dense=False, emb_trainable=False):

    x_input = Input(shape=(input_shape,))

    emb = Embedding(num_words, emb_size, weights=[emb_matrix], trainable=emb_trainable, name='embs')(x_input)
    emb = SpatialDropout1D(0.3)(emb)
#     rnn = SpatialDropout1D(0.15)(rnn)
    
    cnn1 = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(emb)
    cnn2 = Conv1D(filters=64, kernel_size=4, activation='relu', padding='same')(emb)
    cnn3 = Conv1D(filters=64, kernel_size=5, activation='relu', padding='same')(emb)
#     cnn4 = Conv1D(filters=64, kernel_size=6, activation='relu', padding='same')(emb)
    
    x = concatenate([cnn1, cnn2, cnn3])
    x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
    
    if attention == 1: x = AttentionWeightedAverage()(x)
    elif attention == 2: x = Attention()(x)
    else: x = GlobalMaxPooling1D()(x)
    
    x = Dropout(0.3)(x)
    
    if dense: 
        x = Dense(32, activation='relu')(x)
        x = Dropout(0.3)(x)
    
    x_output = Dense(classes, activation='sigmoid')(x)
    return Model(inputs=x_input, outputs=x_output)

In [13]:
# MaxPool2D
def get2xBiCuDNNGRUMaxPool2DModel(input_shape, classes, num_words, emb_size, emb_matrix,
                                attention=0, dense=False, emb_trainable=False):

    x_input = Input(shape=(input_shape,))
    
    emb = Embedding(num_words, emb_size, weights=[emb_matrix], trainable=emb_trainable, name='embs')(x_input)
    emb = SpatialDropout1D(0.3)(emb)
        
    rnn1 = Bidirectional(CuDNNGRU(64, return_sequences=True))(emb)
    rnn2 = Bidirectional(CuDNNGRU(64, return_sequences=True))(rnn1)
    x = concatenate([rnn1, rnn2])

    if attention == 1: x1 = AttentionWeightedAverage()(x)
    elif attention == 2: x1 = Attention()(x)
    else: x1 = GlobalMaxPooling1D()(x)
        
    if attention == 1: x2 = AttentionWeightedAverage()(Permute((2, 1))(x))
    elif attention == 2: x2 = Attention()(Permute((2, 1))(x))
    else: x2 = GlobalMaxPooling1D()(Permute((2, 1))(x))
        
    x = concatenate([x1, x2])
    
    if dense: 
        x = Dense(32, activation='relu')(x)
        x = Dropout(0.3)(x)
    
    x_output = Dense(classes, activation='sigmoid')(x)
    return Model(inputs=x_input, outputs=x_output)

In [14]:
# VDCNN
from keras.models import Model
from keras.layers.convolutional import Conv1D
from keras.layers.embeddings import Embedding
from keras.layers import Input, Dense, Dropout, Lambda, Activation
from keras.layers.pooling import MaxPooling1D
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
import tensorflow as tf

num_filters_default = [64,128,256] # from VDCNN paper


def VDCNN_model(input_shape,num_classes,num_words,emb_size,emb_matrix,num_filters=num_filters_default,top_k=8,emb_trainable=False):

    inputs = Input(shape=(input_shape, ), dtype='int32', name='inputs')

    embedded_sent = Embedding(num_words, emb_size, weights=[emb_matrix], trainable=emb_trainable, name='embs')(inputs)
    embedded_sent = SpatialDropout1D(0.3)(embedded_sent)
    
    conv = Conv1D(filters=64, kernel_size=3, strides=2, padding="same")(embedded_sent)

    for i in range(len(num_filters)):
        conv = ConvBlockVDCNN(conv.get_shape().as_list()[1:], num_filters[i])(conv)
        conv = MaxPooling1D(pool_size=3, strides=2, padding="same")(conv)
        
    def k_max_pooling(x):
        x = tf.transpose(x, [0, 2, 1])
        k_max = tf.nn.top_k(x, k=top_k)
        return tf.reshape(k_max[0], (-1, num_filters[-1] * top_k))

    k_max = Lambda(k_max_pooling,output_shape=(num_filters[-1] * top_k,))(conv)
    k_max = Dropout(0.3)(k_max)

    # fully-connected layers
    fc1 = Dropout(0.3)(Dense(256, activation='relu', kernel_initializer='he_normal')(k_max))
    fc2 = Dropout(0.3)(Dense(128, activation='relu', kernel_initializer='he_normal')(fc1))
    fc3 = Dense(num_classes, activation='sigmoid')(fc2)

    model = Model(inputs=inputs, outputs=fc3)
    return model

class ConvBlockVDCNN(object):

    def __init__(self, input_shape, num_filters):
        self.model = Sequential()
        self.model.add(Conv1D(filters=num_filters, kernel_size=3, strides=1, padding="same", input_shape=input_shape))
        self.model.add(BatchNormalization())
        self.model.add(Activation('relu'))

        self.model.add(Conv1D(filters=num_filters, kernel_size=3, strides=1, padding="same"))
        self.model.add(BatchNormalization())
        self.model.add(Activation('relu'))

    def __call__(self, inputs):
        return self.model(inputs)

In [15]:
# def Conv2D_block(reshape, sequence_length, embedding_dim):
#     filter_sizes = [3,4,5]
#     num_filters = 32

#     conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embedding_dim), padding='valid', kernel_initializer='he_uniform', activation='relu')(reshape)
#     conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embedding_dim), padding='valid', kernel_initializer='he_uniform', activation='relu')(reshape)
#     conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embedding_dim), padding='valid', kernel_initializer='he_uniform', activation='relu')(reshape)

#     maxpool_0 = MaxPool2D(pool_size=(sequence_length - filter_sizes[0] + 1, 1), strides=(1,1), padding='valid')(conv_0)
#     maxpool_1 = MaxPool2D(pool_size=(sequence_length - filter_sizes[1] + 1, 1), strides=(1,1), padding='valid')(conv_1)
#     maxpool_2 = MaxPool2D(pool_size=(sequence_length - filter_sizes[2] + 1, 1), strides=(1,1), padding='valid')(conv_2)

#     concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2])
#     flatten = Flatten()(concatenated_tensor)
#     return flatten


# def Art_CNN(maxlen, max_features, embed_size, embedding_matrix):
#     sequence_input = Input(shape=(maxlen,), dtype='int32')
#     embedding = Embedding(max_features, embed_size, weights=[embedding_matrix],input_length=maxlen,trainable=False)(sequence_input)
#     x = SpatialDropout1D(0.2)(embedding)
#     reshape = Reshape((maxlen,embed_size,1))(x)
#     x = Conv2D_block(reshape,maxlen,embed_size)
#     x = Dense(256, activation="relu")(x)
#     x = Dropout(0.2)(x)
#     preds = Dense(6, activation='sigmoid')(x)
#     model = Model(sequence_input, preds)
#     return model

In [16]:
# Current exp model
def getExpModel0(input_shape, classes, num_words, emb_size, emb_matrix, emb_dropout=0.5,
                 attention=0, dense=False, emb_trainable=False):

    x_input = Input(shape=(input_shape,))
    
    emb = Embedding(num_words, emb_size, weights=[emb_matrix], trainable=emb_trainable, name='embs')(x_input)
    emb = SpatialDropout1D(emb_dropout)(emb)
        
    rnn1 = Bidirectional(CuDNNGRU(64, return_sequences=True))(emb)
    rnn2 = Bidirectional(CuDNNGRU(64, return_sequences=True))(rnn1)
    x = concatenate([rnn1, rnn2])

    if attention == 1: x = AttentionWeightedAverage()(x)
    elif attention == 2: x = Attention()(x)
    else: x = GlobalMaxPooling1D()(x)
    
    if dense: 
        x = Dense(32, activation='relu')(x)
        x = Dropout(0.3)(x)
    
    x_output = Dense(classes, activation='sigmoid')(x)
    return Model(inputs=x_input, outputs=x_output)

In [17]:
# import importlib
# importlib.reload(toxic.models)
# from toxic.models import *

model_name = 'exp_model'

model = getModel0(input_shape=X.shape[1], classes=Y.shape[1], num_words=len(text_analyzer.inx2emb), 
                  emb_size=text_analyzer.emb_size, emb_matrix=text_analyzer.emb_vectors,
                  emb_dropout=0.5, attention=0, dense=False, emb_trainable=False, gru=True)

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 450)          0                                            
__________________________________________________________________________________________________
embs (Embedding)                (None, 450, 300)     51871500    input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 450, 300)     0           embs[0][0]                       
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 450, 128)     140544      spatial_dropout1d_1[0][0]        
__________________________________________________________________________________________________
bidirectio

In [18]:
model_checkpoint = ModelCheckpoint(models_dir+model_name+'.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=6, verbose=1, mode='min')
lr_schedule = LearningRateScheduler(lr_change, verbose=1)
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=1, min_lr=0.0001, verbose=1)
# tensorboard = TensorBoard(log_dir='logs', write_graph=False)

In [19]:
batch_size = 256
val_batch_size = 1024
weights = getClassWeights(Y, mu=0.5)

# trn_seq = StratifiedFeatureSequence(X[trn_inx], Y[trn_inx], batch_size)
trn_seq = FeatureSequence(X[trn_inx], Y[trn_inx], batch_size, shuffle=True)
val_seq = FeatureSequence(X[val_inx], Y[val_inx], val_batch_size)
roc_auc_eval = RocAucEvaluation(X[val_inx], Y[val_inx], batch_size=val_batch_size)

In [20]:
epochs=32
def focal_loss_keras(y_true, y_pred):
    return focal_loss(y_true, y_pred, 1.0, 0.25)

clr = CyclicLR(base_lr=0.0001, max_lr=0.003, step_size=2*len(trn_seq), mode='triangular2')
model.compile(loss="binary_crossentropy", optimizer=optimizers.Nadam())
# model.compile(loss=focal_loss_keras, optimizer=optimizers.Nadam())
# model.compile(loss=art_loss, optimizer=optimizers.Nadam())

model.fit_generator(
    generator=trn_seq, steps_per_epoch=len(trn_seq),
    validation_data=val_seq, validation_steps=len(val_seq),
    initial_epoch=0, epochs=epochs, shuffle=False, verbose=1,
    callbacks=[roc_auc_eval, model_checkpoint, clr, early_stop],
#     callbacks=[roc_auc_eval, model_checkpoint, lr_reduce, early_stop],
    use_multiprocessing=False, workers=cpu_cores, max_queue_size=8*cpu_cores)

Epoch 1/32
561/561 [==============================] - 88s 157ms/step - loss: 0.1280 - val_loss: 0.0473
ROC-AUC: 0.96839956



Epoch 00001: val_loss improved from inf to 0.04735, saving model to models/exp_model.h5
Epoch 2/32
561/561 [==============================] - 88s 157ms/step - loss: 0.0508 - val_loss: 0.0424
ROC-AUC: 0.98194927



Epoch 00002: val_loss improved from 0.04735 to 0.04241, saving model to models/exp_model.h5
Epoch 3/32
561/561 [==============================] - 88s 158ms/step - loss: 0.0449 - val_loss: 0.0400
ROC-AUC: 0.98761452



Epoch 00003: val_loss improved from 0.04241 to 0.04003, saving model to models/exp_model.h5
Epoch 4/32
561/561 [==============================] - 88s 158ms/step - loss: 0.0414 - val_loss: 0.0382
ROC-AUC: 0.98925409



Epoch 00004: val_loss improved from 0.04003 to 0.03816, saving model to models/exp_model.h5
Epoch 5/32
561/561 [==============================] - 88s 158ms/step - loss: 0.0402 - val_loss: 0.0382
ROC-AUC: 0.98961268



Epoch 

In [21]:
# matplotlib.rcParams['figure.figsize'] = (24,8)

# plt.xlabel('Learning Rate')
# plt.ylabel('Loss')
# plt.plot(clr.history['lr'], clr.history['loss'])

In [22]:
del model
model = load_model(models_dir+model_name+'.h5', compile=True, 
                   custom_objects={'Attention':Attention, 'AttentionWeightedAverage':AttentionWeightedAverage, 
                                   'focal_loss_keras':focal_loss_keras})

In [23]:
Y_trn_pred = model.predict(X[trn_inx], batch_size=1024, verbose=0)
losses = compute_losses(Y[trn_inx], Y_trn_pred, eps=1e-5)
for label, label_loss in zip(inx2label, losses):
    print("{}: {}".format(label, label_loss))
print()

trn_loss = sum(losses)/len(losses)
trn_auc = metrics.roc_auc_score(Y[trn_inx], Y_trn_pred)

print()
print("avg_loss: {}".format(trn_loss))
print("ROC AUC: {}".format(trn_auc))

toxic: 0.0644790639332429
severe_toxic: 0.018332696792754948
obscene: 0.03288714078080629
threat: 0.005449422432136256
insult: 0.04469190897774125
identity_hate: 0.014778284843220347


avg_loss: 0.030103086293317
ROC AUC: 0.9945767464582523


In [24]:
Y_val_pred = model.predict(X[val_inx], batch_size=1024, verbose=0)
losses = compute_losses(Y[val_inx], Y_val_pred, eps=1e-5)
for label, label_loss in zip(inx2label, losses):
    print("{}: {}".format(label, label_loss))
print()

val_loss = sum(losses)/len(losses)
val_auc = metrics.roc_auc_score(Y[val_inx], Y_val_pred)

print()
print("avg_loss: {}".format(val_loss))
print("ROC AUC: {}".format(val_auc))

toxic: 0.08041645608308227
severe_toxic: 0.019020405890425866
obscene: 0.03850926609167262
threat: 0.006716398895765416
insult: 0.05601283061553505
identity_hate: 0.01628399130231081


avg_loss: 0.03615989147979867
ROC AUC: 0.9914585334927861


In [25]:
stop

NameError: name 'stop' is not defined

In [ ]:
epochs1=epochs+8
clr = CyclicLR(base_lr=0.0001, max_lr=0.001, step_size=2*len(trn_seq), mode='triangular2')
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=4, verbose=1, mode='auto')
model.compile(loss="binary_crossentropy", optimizer=optimizers.Adam())
# model.compile(loss=art_loss, optimizer=optimizers.Adam())
model.fit_generator(
    generator=trn_seq, steps_per_epoch=len(trn_seq),
    validation_data=val_seq, validation_steps=len(val_seq),
    initial_epoch=epochs, epochs=epochs1, shuffle=False, verbose=1,
#     class_weight=weights,
    callbacks=[model_checkpoint, clr, early_stop, roc_auc_eval],
#     callbacks=[model_checkpoint, lr_reduce, early_stop, roc_auc_eval],
    use_multiprocessing=False, workers=cpu_cores, max_queue_size=8*cpu_cores)

In [ ]:
del model
model = load_model(models_dir+model_name+'.h5', compile=True, 
                   custom_objects={'Attention':Attention, 'AttentionWeightedAverage':AttentionWeightedAverage, 'art_loss':art_loss})

In [ ]:
Y_val_pred = model.predict(X[val_inx], batch_size=1024, verbose=0)
losses = compute_losses(Y[val_inx], Y_val_pred, eps=1e-5)
for label, label_loss in zip(inx2label, losses):
    print("{}: {}".format(label, label_loss))
print()

val_loss = sum(losses)/len(losses)
val_auc = metrics.roc_auc_score(Y[val_inx], Y_val_pred)

print()
print("avg_loss: {}".format(val_loss))
print("ROC AUC: {}".format(val_auc))

In [ ]:
stop

In [ ]:
# np.save("data/new_preprocessing_Y_val_pred.npy", Y_val_pred)
# np.save("data/new_preprocessing_val_inx.npy", val_inx)
# FileLink("data/new_preprocessing_val_inx.npy")

In [ ]:
submission_name = 'exp__focal_loss_a1.0_g0.5__fasttext__d0.3__submission_'+str(round(val_loss, 5))+'_'+str(round(val_auc, 5))+'.csv'

sample_submission = pd.read_csv(data_dir + 'sample_submission.csv')
test_pred = model.predict(test_X, batch_size=1024, verbose=1)
sample_submission[inx2label] = test_pred
sample_submission.to_csv(results_dir+submission_name, index=False)

FileLink(results_dir+submission_name)

In [ ]:
# pseudo
stop

In [ ]:
model_loss_checkpoint = ModelCheckpoint(models_dir+model_name+'_pseudo.h5', monitor='val_loss', verbose=1, mode='min', save_best_only=True)
model.compile(optimizer=optimizers.Nadam(0.0001), loss='binary_crossentropy')

ps_epochs = 3
for ps_inx in range(0, ps_epochs):  
    test_Y = model.predict(test_X, batch_size=1024, verbose=1)
    
    trn_ps_seq = PseudoFeatureSequence(X[trn_inx], X_meta[trn_inx], Y[trn_inx], 182, 
                                       test_X, np.zeros((test_X.shape[0], 2)), test_Y, 74,  
                                       shuffle=True)
    model.fit_generator(
        generator=trn_ps_seq, steps_per_epoch=len(trn_ps_seq),  
        validation_data=val_seq, validation_steps=len(val_seq),
        initial_epoch=epochs+ps_inx, epochs=epochs+ps_inx+1, 
        shuffle=False, verbose=1,
        class_weight=weights,
        callbacks=[model_loss_checkpoint, roc_auc_eval],
        use_multiprocessing=False, workers=cpu_cores, max_queue_size=4*cpu_cores)

In [ ]:
Y_val_pred = model.predict(X[val_inx], batch_size=512, verbose=0)
losses = compute_losses(Y[val_inx], Y_val_pred, eps=1e-5)

val_loss = sum(losses)/len(losses)
val_auc = metrics.roc_auc_score(Y[val_inx], Y_val_pred)

print()
print("avg_loss: {}".format(val_loss))
print("ROC AUC: {}".format(val_auc))

In [ ]:
submission_name = 'fasttext__gru__max_pool2d__submission_'+str(round(val_loss, 5))+'_'+str(round(val_auc, 5))+'_pseudo'+ps_epochs+'.csv'

sample_submission = pd.read_csv(data_dir + 'sample_submission.csv')
test_pred = model.predict(test_X, batch_size=1024, verbose=1)
sample_submission[inx2label] = test_pred
sample_submission.to_csv(results_dir+submission_name, index=False)

FileLink(results_dir+submission_name)

In [ ]:
stop

In [ ]:
# oov_word_hits = {}
# oov_doc_hits = {}
# for w in text_analyzer._oov_vec.keys():
#     oov_word_hits[w] = text_analyzer.word_hits[w]
#     oov_doc_hits[w] = text_analyzer.doc_hits[w]

# pickle.dump(oov_word_hits, open('data/oov_word_hits.pkl', 'wb'))
# pickle.dump(oov_doc_hits, open('data/oov_doc_hits.pkl', 'wb'))